# Processamento de imagens

Todas estas bibliotecas são essenciais para o funcionamento do código a seguir. Nesse código, em especial, é trabalhada a extração de características de imagens, transformadas em dados para posterior análise e retirada de insights.

# Importando as bibliotecas

- greycomatrix: Link - https://scikit-image.org/docs/dev/api/skimage.feature.html#graycomatrix
- greycoprops: Link - https://scikit-image.org/docs/dev/api/skimage.feature.html#graycoprops

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import cv2 
import os
from PIL import Image as im
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops

# Extraindo imagens de um np.array

As imagens utilizadas de um tomógrafo foram cedidas por meio de um arquivo "npy", sendo representadas através de uma matriz [512, 512, 281], correspondendo à 281 imagens. O código abaixo ler o arquivo "npy" e converte-as para imagens.

In [ ]:
imagem = np.load('tc.npy')

for i in range(0, 281):
    img = im.fromarray(np.uint8(imagem[:,:,i]))
    img.save('CT_Scan/{}.png'.format(i))

# Criando as variáveis necessárias e recebendo as imagens

Todas as imagens passam por um processo de transformação para serem trabalhadas pelos algoritmos corretos. Nesse trecho de código, a escala de cores é ajustada e o tamanho das imagens.

In [ ]:
dataset_dir = "CT_Scan"

imgs = [] # Matrix de imagens
show_images = [] # Imagens para serem exibidas sem tratamento
labels = [] # Nome de cada imagem

for i, filename in enumerate(os.listdir(dataset_dir)):
    label = filename
    filename = os.path.join(dataset_dir, filename)
    img = cv2.imread(filename)
    show_images.append(img)

    # Tratamento da imagem para trabalharmos
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
    crop = gray[ymin:ymax, xmin:xmax]
    resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)

    imgs.append(resize)
    labels.append(label)

# Exibindo algumas imagens

Para melhor visualizarmos e termos noção das imagens que estramos trabalhando, o trecho de código a seguir é utilizado para exibir algumas imagens. Somente como um demonstrativo.

In [ ]:
plt.figure(figsize= (20, 20))

for i in range(0, 25):
    plt.subplot(5, 5, i + 1)
    image = show_images[i] / 255       # scales data to range (0 - 255)
    plt.imshow(image)
    img_title = labels[i]   # get class of image
    plt.title(img_title, color= 'blue', fontsize= 12)
    plt.axis('off')
plt.show()

# Gray-Level Co-ocurrence Matrix

Um método estatístico de examinar a textura que considera a relação espacial de pixels é a Gray-Level Co-ocurrence Matrix (GLCM), também conhecida como matriz de dependência espacial de nível de cinza. As funções GLCM caracterizam a textura de uma imagem calculando com que frequência pares de pixels com valores específicos e em uma relação espacial especificada ocorrem em uma imagem, criando um GLCM e, em seguida, extraindo medidas estatísticas dessa matriz. As funções de filtro de textura, não podem fornecer informações sobre a forma.

Depois de criar os GLCMs usando a graycomatrix, você pode derivar várias estatísticas deles usando graycoprops. Essas estatísticas fornecem informações sobre a textura de uma imagem.

O código a seguir é responsável por parametrizar o algoritmo GLCM e executá-lo.

In [ ]:
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_features = [] # Cada uma das propriedades é trazida para a lista de features

for img, label in zip(imgs, labels):
    glcm = greycomatrix(img, distances=[5], angles=[0, np.pi/2], levels=256, symmetric=True, normed=True)
    feature = []
    glcm_props = [propery for name in properties for propery in greycoprops(glcm, name)[0]]

    for item in glcm_props:
            feature.append(item)

    glcm_features.append(feature)
 
columns = []
angles = ['0', '90']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
# columns.append("label")

# Verificando como ficou a base de dados

Após a extração de features, vamos verificar como ficou a base de dados. Somente como um demonstrativo.

In [ ]:
glcm_df = pd.DataFrame(glcm_features, columns = columns)

glcm_df.head()

# Resumo estatístico

A função describe() fornece um resumo estatístico das principais estimativas de localização e variabilidade.

In [ ]:
glcm_df.describe()

# Histograma

Um histograma é um gráfico usado para representar a distribuição de frequência de alguns pontos de dados de uma variável. Os histogramas geralmente classificam os dados em vários “compartimentos/bins” ou “grupos de intervalo/range groups” e contam quantos pontos de dados pertencem a cada um desses compartimentos.

In [ ]:
plt.figure()
glcm_df.hist(figsize=(16, 14))
plt.show()

# Scatter Matrix

Gráficos de dispersão são usados para mostrar relacionamentos. Para correlação, os gráficos de dispersão ajudam a mostrar a força da relação linear entre duas variáveis. Para regressão, os gráficos de dispersão geralmente adicionam uma linha ajustada. No controle de qualidade, os gráficos de dispersão geralmente incluem limites de especificação ou linhas de referência.

In [ ]:
sns.pairplot(glcm_df)